In [1]:
import pandas as pd

## A/B Testing For ShoeFly.com

In [2]:
ad_clicks = pd.read_csv('ad_clicks.csv')

ad_clicks.head(5)

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


In [3]:
ad_clicks.groupby('utm_source').user_id.count().reset_index()

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


In [4]:
ad_clicks[ad_clicks.ad_click_timestamp.isnull()].head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A


In [5]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

In [6]:
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [7]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False      175
1      email      True       80
2   facebook     False      324
3   facebook      True      180
4     google     False      441
5     google      True      239
6    twitter     False      149
7    twitter      True       66


In [8]:
clicks_pivot = clicks_by_source.pivot(
    columns = 'is_click',
    index = 'utm_source',
    values = 'user_id'
).reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


In [9]:
clicks_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
utm_source    4 non-null object
False         4 non-null int64
True          4 non-null int64
dtypes: int64(2), object(1)
memory usage: 176.0+ bytes


In [10]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])

In [11]:
clicks_pivot.head()

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


In [12]:
ad_clicks.groupby('experimental_group').user_id.count()

experimental_group
A    827
B    827
Name: user_id, dtype: int64

In [13]:
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


In [14]:
ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count()

experimental_group  is_click
A                   False       517
                    True        310
B                   False       572
                    True        255
Name: user_id, dtype: int64

In [15]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']
a_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False


In [16]:
a_clicks_stage = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
a_clicks_pivot = a_clicks_stage.pivot(
    columns = 'is_click',
    index = 'day',
    values = 'user_id'
).reset_index()

a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False])

print(a_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     70    43         0.380531
1           2 - Tuesday     76    43         0.361345
2         3 - Wednesday     86    38         0.306452
3          4 - Thursday     69    47         0.405172
4            5 - Friday     77    51         0.398438
5          6 - Saturday     73    45         0.381356
6            7 - Sunday     66    43         0.394495


In [17]:
b_clicks_stage = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index()
b_clicks_pivot = b_clicks_stage.pivot(
    columns = 'is_click',
    index = 'day',
    values = 'user_id'
).reset_index()

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])

print(b_clicks_pivot)

is_click            day  False  True  percent_clicked
0            1 - Monday     81    32         0.283186
1           2 - Tuesday     74    45         0.378151
2         3 - Wednesday     89    35         0.282258
3          4 - Thursday     87    29         0.250000
4            5 - Friday     90    38         0.296875
5          6 - Saturday     76    42         0.355932
6            7 - Sunday     75    34         0.311927


## Working with Multiple Tables in Pandas

In [24]:
orders = pd.read_csv('wk_2/orders.csv')
print(orders)
products = pd.read_csv('wk_2/products.csv')
print(products)
customers = pd.read_csv('wk_2/customers.csv')
print(customers)

   order_id  customer_id  product_id  quantity   timestamp
0         1            2           3         1  2017-01-01
1         2            2           2         3  2017-01-01
2         3            3           1         1  2017-01-01
3         4            3           2         2  2017-02-01
4         5            3           3         3  2017-02-01
5         6            1           4         2  2017-03-01
6         7            1           1         1  2017-02-02
7         8            1           4         1  2017-02-02
   product_id         description  price
0           1      thing-a-ma-jig      5
1           2  whatcha-ma-call-it     10
2           3          doo-hickey      7
3           4               gizmo      3
   customer_id customer_name        address  phone_number
0            1    John Smith   123 Main St.  212-123-4567
1            2      Jane Doe  456 Park Ave.  949-867-5309
2            3     Joe Schmo   798 Broadway  112-358-1321


In [32]:
print(orders[orders.order_id == 3])
print(products[products.product_id == 1].description)

   order_id  customer_id  product_id  quantity   timestamp
2         3            3           1         1  2017-01-01
0    thing-a-ma-jig
Name: description, dtype: object


In [30]:
print(products[products.product_id == 1].description)

0    thing-a-ma-jig
Name: description, dtype: object

#### Inner Merge II
It is easy to do this kind of matching for one row, but hard to do it for multiple rows.

Luckily, Pandas can efficiently do this for the entire table. We use the .merge method.

The .merge method looks for columns that are common between two DataFrames and then looks for rows where those column's values are the same. It then combines the matching rows into a single row in a new table.

We can call the pd.merge method with two tables like this:

new_df = pd.merge(orders, customers)

---
#### Inner Merge III
In addition to using pd.merge, each DataFrame has its own merge method. For instance, if you wanted to merge orders with customers, you could use:

new_df = orders.merge(customers)
This produces the same DataFrame as if we had called pd.merge(orders, customers).

We generally use this when we are joining more than two DataFrames together because we can "chain" the commands. The following command would merge orders to customers, and then the resulting DataFrame to products:

big_df = orders.merge(customers)\
    .merge(products)

In [ ]:
sales = pd.read_csv('sales.csv')
print sales
targets = pd.read_csv('targets.csv')
print targets

men_women = pd.read_csv('men_women_sales.csv')

all_data = sales.merge(targets).merge(men_women)

print(all_data)

results = all_data[(all_data.revenue > all_data.target) & (all_data.women > all_data.men)]

print(results)

In [ ]:
pd.merge(
    orders,
    customers.rename(columns={'id': 'customer_id'}))

In [ ]:
Merge on Specific Columns II
In the previous exercise, we learned how to use rename to merge two DataFrames whose columns don't match.

If we don''t want to do that, we have another option. We could use the keywords left_on and right_on to specify which columns we want to perform the merge on. In the example below, the "left" table is the one that comes first (orders), and the "right" table is the one that comes second (customers). This syntax says that we should match the customer_id from orders to the id in customers.

pd.merge(
    orders,
    customers,
    left_on='customer_id',
    right_on='id')

In [ ]:
The new column names id_x and id_y aren''t very helpful for us when we read the table. We can help make them more useful by using the keyword suffixes. We can provide a list of suffixes to use instead of "_x" and "_y".

For example, we could use the following code to make the suffixes reflect the table names:

pd.merge(
    orders,
    customers,
    left_on='customer_id',
    right_on='id',
    suffixes=['_order', '_customer'])

In [ ]:
orders_products = pd.merge(
  orders,
  products,
  left_on='product_id',
  right_on='id',
  suffixes=['_orders', '_products']
)

#### Outer Merge
In the previous exercise, we saw that when we merge two DataFrames whose 
rows don''t match perfectly, we lose the unmatched rows.

This type of merge (where we only include matching rows) is called an inner merge. 
There are other types of merges that we can use when we want to keep information from the unmatched rows.

If we wanted to combine the data from both companies without losing the customers who are missing from one of the tables, we could use an Outer Join. An Outer Join would include all rows from both tables, even if they don't match. Any missing values are filled in with None or nan (which stands for "Not a Number").

pd.merge(company_a, company_b, how='outer')

#### Left Merge
Let's return to the merge of Company A and Company B.

Suppose we want to identify which customers are missing phone information. We would want a list of all customers who have email, but don't have phone.

We could get this by performing a Left Merge. A Left Merge includes all rows from the first (left) table, but only rows from the second (right) table that match the first table.

For this command, the order of the arguments matters. If the first DataFrame is company_a and we do a left join, we'll only end up with rows that appear in company_a.

pd.merge(company_a, company_b, how='left')

#### Concatenate DataFrames
Sometimes, a dataset is broken into multiple tables. For instance, data is often split into multiple CSV files so that each download is smaller.

When we need to reconstruct a single DataFrame from multiple smaller DataFrames, we can use the method pd.concat([df1, df2, df2, ...]). This method only works if all of the columns are the same in all of the DataFrames.

In [38]:
visits = pd.read_csv('wk_2/visits.csv',
                        parse_dates=[1])
checkouts = pd.read_csv('wk_2/checkouts.csv',
                        parse_dates=[1])

In [39]:
visits.head()

,user_id,visit_time
0,319350b4-9951-47ef-b3a7-6b252099905f,2017-02-21 07:16:00
1,7435ec9f-576d-4ebd-8791-361b128fca77,2017-05-16 08:37:00
2,0b061e73-f709-42fa-8d1a-5f68176ff154,2017-04-12 19:32:00
3,9133d6f0-e68b-4c8d-bafd-ff2825e8dafe,2017-08-18 04:32:00
4,08d13edb-071c-4cfb-9ee4-8f377d0e932a,2017-07-08 06:24:00


In [40]:
checkouts.head()

,user_id,checkout_time
0,fe90a9f4-960a-4a0d-9160-e562adb79365,2017-11-09 09:25:00
1,1a35b7eb-f603-407d-91be-a2c3304066fd,2017-08-15 21:25:00
2,e2c24ee0-7fdf-4400-abde-b36378fe5ce6,2017-07-04 15:39:00
3,10dbd3c5-d610-44e9-9994-110a7950b6b4,2017-08-09 21:07:00
4,f028e9dd-77d0-4002-83f6-372a4837fda6,2017-10-27 08:57:00


In [41]:
v_to_c = pd.merge(visits, checkouts)

In [42]:
v_to_c.head()

,user_id,visit_time,checkout_time
0,319350b4-9951-47ef-b3a7-6b252099905f,2017-02-21 07:16:00,2017-02-21 07:27:00
1,319350b4-9951-47ef-b3a7-6b252099905f,2017-02-21 07:16:00,2017-02-21 07:40:00
2,7435ec9f-576d-4ebd-8791-361b128fca77,2017-05-16 08:37:00,2017-05-16 08:49:00
3,7435ec9f-576d-4ebd-8791-361b128fca77,2017-05-16 08:37:00,2017-05-16 08:55:00
4,08d13edb-071c-4cfb-9ee4-8f377d0e932a,2017-07-08 06:24:00,2017-07-08 06:32:00


In [43]:
v_to_c['time'] = v_to_c.checkout_time - v_to_c.visit_time

In [44]:
v_to_c.head()

,user_id,visit_time,checkout_time,time
0,319350b4-9951-47ef-b3a7-6b252099905f,2017-02-21 07:16:00,2017-02-21 07:27:00,00:11:00
1,319350b4-9951-47ef-b3a7-6b252099905f,2017-02-21 07:16:00,2017-02-21 07:40:00,00:24:00
2,7435ec9f-576d-4ebd-8791-361b128fca77,2017-05-16 08:37:00,2017-05-16 08:49:00,00:12:00
3,7435ec9f-576d-4ebd-8791-361b128fca77,2017-05-16 08:37:00,2017-05-16 08:55:00,00:18:00
4,08d13edb-071c-4cfb-9ee4-8f377d0e932a,2017-07-08 06:24:00,2017-07-08 06:32:00,00:08:00


In [45]:
print(v_to_c.time.mean())

0 days 00:15:24.750000


In [46]:
v_to_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 4 columns):
user_id          80 non-null object
visit_time       80 non-null datetime64[ns]
checkout_time    80 non-null datetime64[ns]
time             80 non-null timedelta64[ns]
dtypes: datetime64[ns](2), object(1), timedelta64[ns](1)
memory usage: 5.6+ KB


## Page Visits Funnel Project

Cool T-Shirts Inc. has asked you to analyze data on visits to their website. Your job is to build a funnel, which is a description of how many people continue to the next step of a multi-step process.

In this case, our funnel is going to describe the following process:

- A user visits CoolTShirts.com
- A user adds a t-shirt to their cart
- A user clicks "checkout"
- A user actually purchases a t-shirt

In [2]:
visits = pd.read_csv('wk_2_funnel/visits.csv', parse_dates=[1])
cart = pd.read_csv('wk_2_funnel/cart.csv', parse_dates=[1])
checkout = pd.read_csv('wk_2_funnel/checkout.csv', parse_dates=[1])
purchase = pd.read_csv('wk_2_funnel/purchase.csv', parse_dates=[1])

In [3]:
visits.head()

,user_id,visit_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00


In [4]:
cart.head()

,user_id,cart_time
0,2be90e7c-9cca-44e0-bcc5-124b945ff168,2017-11-07 20:45:00
1,4397f73f-1da3-4ab3-91af-762792e25973,2017-05-27 01:35:00
2,a9db3d4b-0a0a-4398-a55a-ebb2c7adf663,2017-03-04 10:38:00
3,b594862a-36c5-47d5-b818-6e9512b939b3,2017-09-27 08:22:00
4,a68a16e2-94f0-4ce8-8ce3-784af0bbb974,2017-07-26 15:48:00


In [5]:
checkout.head()

,user_id,checkout_time
0,d33bdc47-4afa-45bc-b4e4-dbe948e34c0d,2017-06-25 09:29:00
1,4ac186f0-9954-4fea-8a27-c081e428e34e,2017-04-07 20:11:00
2,3c9c78a7-124a-4b77-8d2e-e1926e011e7d,2017-07-13 11:38:00
3,89fe330a-8966-4756-8f7c-3bdbcd47279a,2017-04-20 16:15:00
4,3ccdaf69-2d30-40de-b083-51372881aedd,2017-01-08 20:52:00


In [6]:
purchase.head()

,user_id,purchase_time
0,4b44ace4-2721-47a0-b24b-15fbfa2abf85,2017-05-11 04:25:00
1,02e684ae-a448-408f-a9ff-dcb4a5c99aac,2017-09-05 08:45:00
2,4b4bc391-749e-4b90-ab8f-4f6e3c84d6dc,2017-11-20 20:49:00
3,a5dbb25f-3c36-4103-9030-9f7c6241cd8d,2017-01-22 15:18:00
4,46a3186d-7f5a-4ab9-87af-84d05bfd4867,2017-06-11 11:32:00


In [7]:
visits_cart = pd.merge(visits, cart, how='left')
visits_cart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2052 entries, 0 to 2051
Data columns (total 3 columns):
user_id       2052 non-null object
visit_time    2052 non-null datetime64[ns]
cart_time     400 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 64.1+ KB


In [9]:
len(visits_cart[visits_cart.cart_time.isnull() == True])

1652

In [11]:
float((1652/2052) * 100)

80.50682261208577

In [12]:
print((float(1652)/2052) * 100)

80.50682261208577


In [13]:
cart_checkout = pd.merge(cart, checkout, how='left')
cart_checkout.info()
print(len(cart_checkout[cart_checkout.checkout_time.isnull() == True]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 0 to 601
Data columns (total 3 columns):
user_id          602 non-null object
cart_time        602 non-null datetime64[ns]
checkout_time    476 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 18.8+ KB
126


In [14]:
print((float(126)/602) * 100)

20.930232558139537


In [15]:
print((float(476)/602) * 100)

79.06976744186046


In [16]:
all_data = visits.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left')

In [17]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2594 entries, 0 to 2593
Data columns (total 5 columns):
user_id          2594 non-null object
visit_time       2594 non-null datetime64[ns]
cart_time        942 non-null datetime64[ns]
checkout_time    816 non-null datetime64[ns]
purchase_time    696 non-null datetime64[ns]
dtypes: datetime64[ns](4), object(1)
memory usage: 121.6+ KB


In [18]:
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:49:00,NaT,NaT
4,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT


In [23]:
print(len(all_data[(all_data.checkout_time.isnull() == False) & (all_data.purchase_time.isnull() == True)]))

120


In [24]:
print((float(126)/816) * 100)

15.441176470588236


In [25]:
print((float(696)/816) * 100)

85.29411764705883


In [26]:
all_data['time_to_purchase'] = all_data.purchase_time - all_data.visit_time

In [27]:
all_data.time_to_purchase.mean()

Timedelta('0 days 00:44:02.672413')